In [ ]:
import requests
import json
from hazm import Normalizer, word_tokenize, stopwords_list
from collections import defaultdict
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# لیست حروف اضافه فارسی و کلمات اضافی
stop_words = set(stopwords_list())
custom_stop_words = set(['سلام', 'ببخشید', 'لطفا', 'گروه'])  # کلمات مورد نظر خود را اینجا اضافه کنید

# ترکیب دو لیست
all_stop_words = stop_words.union(custom_stop_words)


########################
def read_json_file(file_path):
  """Reads a JSON file and returns its content as a Python dictionary."""
  with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
  return data

# Replace 'folder/AKHARINKHABAR.json' with the actual path to your JSON file
file_path = 'your file path.json'
data = read_json_file(file_path)
###################################################
# ابزارهای hazm را راه‌اندازی کنید
normalizer = Normalizer()

# احراز هویت Hugging Face
token = "your Hugging Face token-go to your hugging face account and https://huggingface.co/settings/tokens/new?"

# مدل تحلیل احساسات را بارگیری کنید
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased-sentiment-digikala", token=token)
model = AutoModelForSequenceClassification.from_pretrained("HooshvareLab/bert-fa-base-uncased-sentiment-digikala", token=token)

# شمارش پیام‌های هر کاربر
user_message_counts = defaultdict(int)
user_messages = defaultdict(list)

# توابع تحلیل احساسات
def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return torch.argmax(probs, dim=-1).item(), probs.max().item()

sentiment_labels = ["منفی", "خنثی", "مثبت"]

# تحلیل احساسات پیام‌ها و جمع‌بندی احساسات هر کاربر
user_sentiments = defaultdict(list)

for message in data['messages']:
    if 'from' in message and 'text' in message:
        user_id = message['from']
        if isinstance(message['text'], list):
            text = ' '.join([str(item) for item in message['text'] if isinstance(item, str)])
        else:
            text = str(message['text'])

        normalized_text = normalizer.normalize(text)
        user_message_counts[user_id] += 1
        user_messages[user_id].append(normalized_text)

        # تحلیل احساسات پیام
        sentiment, confidence = get_sentiment(normalized_text)
        user_sentiments[user_id].append((sentiment_labels[sentiment], confidence))

# فیلتر کردن کاربران با بیش از 50 پیام
active_users = {user_id: texts for user_id, texts in user_messages.items() if user_message_counts[user_id] > 1}

# جمع‌بندی احساسات برای کاربران فعال
user_summary = {}

for user_id, sentiments in user_sentiments.items():
    if user_id in active_users:
        sentiment_counts = defaultdict(int)
        for sentiment, _ in sentiments:
            sentiment_counts[sentiment] += 1

        total_messages = sum(sentiment_counts.values())
        user_summary[user_id] = {sentiment: count / total_messages for sentiment, count in sentiment_counts.items()}

# رسم نمودار ورد کلود برای هر کاربر
for user_id, messages in active_users.items():
    # ترکیب پیام‌های کاربر به یک متن طولانی
    combined_text = ' '.join(messages)

    # حذف کلمات توقف
    words = word_tokenize(combined_text)
    filtered_words = [word for word in words if word not in all_stop_words]
    filtered_text = ' '.join(filtered_words)

    # ایجاد ورد کلود
    wordcloud = WordCloud(font_path='/your farsi font path.ttf', width=800, height=400, background_color='white').generate(filtered_text)

    # نمایش ورد کلود
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Word Cloud for User {user_id}')
    plt.show()

    # نمایش جمع‌بندی احساسات
    print(f'Sentiment Summary for User {user_id}:')
    for sentiment, ratio in user_summary[user_id].items():
        print(f'  {sentiment}: {ratio:.2%}')
